In [ ]:
import os 
os.chdir(r'Q:\sachuriga\Sachuriga_Python/quattrocolo-nwb4fp\src')
import pandas as pd
from pynwb import NWBHDF5IO
import os
from pathlib import Path
from pynwb.file import Subject
from nwb4fp.data.helpers import unit_location_ch,convert_h5py_to_picklable
import pynapple as nap
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Output path for the summary table and unit tables
output_path = r'Q:/sachuriga/CR_CA1_paper/tables/'

# Set your base folder path
base_folder = 'S:/Sachuriga/nwb/test4neo/'

# Define animal groups
target_prefixes_control = ['65165', '65091', '63383', '66539', '65622']
target_prefixes_exp = ['65588', '63385', '66538', '66537', '66922']
males = ['65165', '63383', '66539', '63385', '65622']
females = ['65091', '65588', '66538', '66537', '66922']
surgry_sachuriga = ['65165', '63383', '66539', '63385', '65091', '65588', '66538', '66537', '65283']
surgry_ingvild = ['66922', '65622']

# Initialize dictionaries and lists
summary_data = {}
file_data = []  # For files_df
all_units_list = []  # For all_units_df

# Get all NWB files in the directory
all_files = os.listdir(base_folder)
nwb_files = [f for f in all_files if f.endswith('.nwb')]
print(f"Found {len(nwb_files)} NWB files: {nwb_files}")

# Process each NWB file
for nwb_file in nwb_files:
    nwbpath = os.path.join(base_folder, nwb_file)
    print(f"Processing {nwbpath}")

    # Open and modify NWB file for metadata
    with NWBHDF5IO(Path(nwbpath), "a") as io:
        nwb = io.read()
        animal_id = nwbpath.split('/')[-1][:5]
        day = nwbpath.split('/')[-1].split("_")[1]
        session_as = nwbpath.split('_phy')[0][-1]
        if session_as=="A":
            spi =  1.6517
        elif session_as=="B":
            spi = 1.9060
        elif session_as=="C":
            spi = 1.6501


        print(f"Using the session: {session_as} and the spi is: {spi}")
        print(f"Animal ID: {animal_id}")

        # Assign sex
        sex = "M" if animal_id in males else "F" if animal_id in females else "Unknown"

        if animal_id in target_prefixes_control:
            plus:int = 0
        elif animal_id in target_prefixes_exp:
            plus:int = 1

        # Initialize genotype and experimenter
        genotype = None
        experimenter = None

        # Create or update subject
        try:
            subject = Subject(
                subject_id=animal_id,
                age="P90D",
                description="mouse 5",
                species="Mus musculus",
                sex=sex)
            nwb.subject = subject
            
            if animal_id in target_prefixes_control:
                nwb.subject.genotype = "NDNF-flp-/- and Pde1c -/-"
            elif animal_id in target_prefixes_exp:
                nwb.subject.genotype = "NDNF-flp +/- and Pde1c +/-"

            if animal_id in surgry_sachuriga:
                nwb.experimenter = ("Sachuriga",)
            elif animal_id in surgry_ingvild:
                nwb.experimenter = ("Ingvild Lynneberg Glærum",)

            genotype = nwb.subject.genotype
            experimenter = nwb.experimenter[0] if nwb.experimenter else "Unknown"

        except AttributeError:
            print(f"AttributeError for {animal_id}, skipping subject update")
            genotype = nwb.subject.genotype
            experimenter = nwb.experimenter[0] if nwb.experimenter else "Unknown"

        io.write(nwb)

    # Open the NWB file again to process units
    with NWBHDF5IO(Path(nwbpath), "a") as io:
        nwb = io.read()
        unit_table = nwb.units.to_dataframe()
        
        # Initialize counters if animal_id not in summary_data
        if animal_id not in summary_data:
            summary_data[animal_id] = {
                "Animal ID": animal_id,
                "Sex": sex,
                "Genotype": genotype,
                "Experimenter": experimenter,
                "Sessions": 0,
                "Total Units": 0,
                "Good Units": 0,
                "Pyramidal Cells": 0,
                "Narrow Spike Interneurons": 0,
                "Wide Spike Interneurons": 0,
                "Speed Cells": 0,
                "Place Cells": 0
            }

        # Increment session count
        summary_data[animal_id]["Sessions"] += 1

        # Count total units
        total_units = len(unit_table.index)
        summary_data[animal_id]["Total Units"] += total_units

        # Lists to store unit classifications
        unit_quality = []
        cell_type = []
        functional_cell_type = []
        cell_type_group = []
        cell_type_group4 = []
        sessionss=[]
        lfp_unit=[]
        ch=[]
        # Create a copy of unit_table with additional columns
        units_df = unit_table.copy()
        units_df['animal_id'] = animal_id
        units_df['session_id'] = nwb_file

        id = [0, 2, 4]
        # Classify each unit

        scaler = MinMaxScaler(feature_range=(0, 1))
        time = nwb.processing['Behavioral data']['Mid_brain_cords']['Mid_brain_cords'].timestamps[:]
        xy = nwb.processing['Behavioral data']['Mid_brain_cords']['Mid_brain_cords'].data[:]
        pos = np.column_stack((
            time,
            scaler.fit_transform(xy[:, 0].reshape(-1, 1)).flatten(),
            scaler.fit_transform(xy[:, 1].reshape(-1, 1)).flatten()
        ))
        lfp = nwb.processing['lfp_raw']['LFP']['lfp_raw'].data[:]
        lfp_times = nwb.processing['Behavioral data']['Head(snout2neck)_Direction']['lfp_times'].data[:]


        for i in range(len(unit_table.index)):
            sessionss.append(session_as)
            unit_x = nwb.units[i]['x'].values[0]
            unit_y = nwb.units[i]['y'].values[0]
            ch.append(unit_location_ch(file_path=r"Q:\sachuriga\Sachuriga_Python/quattrocolo-nwb4fp/ASSY-236-F.prb",x=unit_x,y=unit_y))
            # lfp_u = lfp[:,ch]
            # lfp_unit.append(np.column_stack((lfp_times, lfp_u)))

            if (unit_table['isi_violations_ratio'][i] <= 0.2 and 
                unit_table['l_ratio'][i] <= 0.05 and 
                unit_table['amplitude_median'][i] <= -40):
                unit_quality.append("good")
                summary_data[animal_id]["Good Units"] += 1
                
                # Classify cell type
                if unit_table['peak_to_valley'][i] <= 0.000425:
                    cell_type.append("Narrow spike interneuron")
                    cell_type_group.append(id[1] + plus)
                    cell_type_group4.append(id[1] + plus)
                    summary_data[animal_id]["Narrow Spike Interneurons"] += 1
                    if unit_table['matlab_speedScores'][i] >= 0.3:
                        functional_cell_type.append("Speed cell")
                        summary_data[animal_id]["Speed Cells"] += 1
                    else:
                        functional_cell_type.append("normal_int")
                elif unit_table['peak_to_valley'][i] > 0.000425 and unit_table['matlab_acg_tau_rise'][i] >= 6:
                    cell_type_group.append(id[2] + plus)
                    cell_type_group4.append(id[1] + plus)
                    cell_type.append("Wide Spike Interneurons")
                    summary_data[animal_id]["Wide Spike Interneurons"] += 1
                    if unit_table['matlab_speedScores'][i] >= 0.3:
                        functional_cell_type.append("Speed cell")
                        summary_data[animal_id]["Speed Cells"] += 1
                    else:
                        functional_cell_type.append("normal_int")
                elif unit_table['peak_to_valley'][i] > 0.000425 and unit_table['matlab_acg_tau_rise'][i] < 6:
                    cell_type_group.append(id[0] + plus)
                    cell_type_group4.append(id[0] + plus)
                    cell_type.append("Pyramidal cells")
                    summary_data[animal_id]["Pyramidal Cells"] += 1
                    if (unit_table['matlab_reject_h0'][i] == 1 and 
                        unit_table['matlab_test_stat_si'][i] >= spi and 
                        unit_table['matlab_maxfsize'][i] >= 20 and 
                        unit_table['firing_range'][i] <= 10):
                        functional_cell_type.append("Place cell")
                        summary_data[animal_id]["Place Cells"] += 1
                    else:
                        functional_cell_type.append("normal_py")
                else:
                    cell_type.append("nah_error")
                    cell_type_group.append("nah_error")
                    cell_type_group4.append("nah_error")
                    functional_cell_type.append("nah_error")
                    unit_quality.append("nah_error")
            else:
                cell_type_group.append("None")
                cell_type_group4.append("None")
                unit_quality.append("bad")
                cell_type.append("None")
                functional_cell_type.append("None")

        # Add classification columns to units_df
        units_df['unit_quality'] = unit_quality
        units_df['cell_type'] = cell_type
        units_df['functional_cell_type'] = functional_cell_type
        units_df['genotype'] = genotype
        units_df['experimenter'] = experimenter
        units_df['sex'] = sex
        units_df['cell_type_group_6_group'] = cell_type_group
        units_df['cell_type_group_2_group'] = cell_type_group4
        units_df['session'] = sessionss
        units_df['ch'] = ch
        # Add file data for files_df
        # file_data.append({
        #     'filename': nwb_file,
        #     'animal_id': animal_id,
        #     'session_id': nwb_file,
        #     'genotype': genotype,
        #     'experimenter': experimenter,
        #     'sex': sex,
        #     'unit_table': units_df,
        #     'pos':pos,
        #     'lfp_times':lfp_times,
        #     'lfp':lfp,
        #     'day': day,
        #     'session': session_as
        #      #'npdata': npdata
        # })
        file_data=[]
        file_data.append({
            'filename': nwb_file,
            'animal_id': animal_id,
            'session_id': nwb_file,
            'genotype': genotype,
            'experimenter': experimenter,
            'sex': sex,
            'unit_table': units_df,
            'pos':pos,
            'lfp_times':lfp_times,
            'lfp':lfp,
            'day': day,
            'session': session_as
             #'npdata': npdata
        })
        files_df = pd.DataFrame(file_data)
        # Add to all_units_list for all_units_df
        output_path = fr"S:\Sachuriga\filre_with_table/"
        print(fr"save to {output_path}")
        all_units_output_file = os.path.join(output_path, fr"{animal_id}_{day}_{session_as}_units_table_withDLC.pkl")
        files_df.to_pickle(all_units_output_file)
        
        # Add columns to the NWB file
        if "unit_quality" not in nwb.units.colnames:
            nwb.units.add_column(name="unit_quality", description="values of unit_quality", data=unit_quality)
        if "cell Type" not in nwb.units.colnames:
            nwb.units.add_column(name="cell Type", description="values of cell_type", data=cell_type)
        if "functional Cell Type" not in nwb.units.colnames:
            nwb.units.add_column(name="functional Cell Type", description="values of functional_cell_type", data=functional_cell_type)

        io.write(nwb)

Found 187 NWB files: ['66922_2024-09-25_15-39-48_A_phy_k_manual.nwb', '63383_2024-07-10_15-37-51_A_phy_k_manual.nwb', '63383_2024-07-13_14-10-05_A_phy_k_manual.nwb', '63383_2024-07-13_14-42-37_B_phy_k_manual.nwb', '63383_2024-07-13_15-10-36_C_phy_k_manual.nwb', '63383_2024-07-15_14-51-19_A_phy_k_manual.nwb', '63383_2024-07-15_15-18-02_B_phy_k_manual.nwb', '63383_2024-07-15_15-44-58_C_phy_k_manual.nwb', '63383_2024-07-18_14-13-10_A_phy_k_manual.nwb', '63383_2024-07-18_14-39-54_B_phy_k_manual.nwb', '63383_2024-07-18_15-07-00_C_phy_k_manual.nwb', '63383_2024-07-20_14-21-42_A_phy_k_manual.nwb', '63383_2024-07-20_14-48-58_B_phy_k_manual.nwb', '63383_2024-07-20_15-16-56_C_phy_k_manual.nwb', '63383_2024-07-22_15-12-01_A_phy_k_manual.nwb', '63383_2024-07-22_15-40-20_B_phy_k_manual.nwb', '63383_2024-07-22_16-10-10_C_phy_k_manual.nwb', '63383_2024-07-25_12-57-40_A_phy_k_manual.nwb', '63383_2024-07-25_13-26-11_B_phy_k_manual.nwb', '63383_2024-07-25_13-53-06_C_phy_k_manual.nwb', '63383_2024-07-28_

ValueError: Per-column arrays must each be 1-dimensional

In [15]:
# Author: Sachuriga
# Email: sachuriga3@gmail.com
# Purpose: Process CSV files and add their columns to corresponding NWB files

import pandas as pd
from pynwb import NWBHDF5IO
import os

target_prefixes_control= ['65165', '65091', '63383', '66539','65622','65283']
target_prefixes_exp = ['65588', '63385', '66538', '66537','66922']


# Set your base folder path
base_folder = 'S:/Sachuriga/nwb/test4neo'

# Get all files in the directory
all_files = os.listdir(base_folder)

# Filter for CSV and NWB files and create pairs
csv_files = [f for f in all_files if f.endswith('.csv')]
nwb_files = [f for f in all_files if f.endswith('.nwb')]

# Process each pair of files
for csv_file in csv_files:
    # Try to find a matching NWB file (assuming similar base names)
    base_name = os.path.splitext(csv_file)[0]
    nwb_file = f"{base_name}.nwb"
    
    if nwb_file in nwb_files:
        csvpath = os.path.join(base_folder, csv_file)
        nwbpath = os.path.join(base_folder, nwb_file)
        
        try:
            # Read the CSV file
            df = pd.read_csv(csvpath)
            
            # Get column names
            column_names = df.columns.tolist()
            
            # Open and modify NWB file
            with NWBHDF5IO(nwbpath, "a") as io:
                nwb = io.read()
                for col in column_names:
                    print(f"Processing column: {col}")
                    nwb.units.add_column(
                        name=f"matlab_{col}",
                        description=f"values of {col}",
                        data=df[col].tolist()
                    )
        
                # Uncomment these if you need to modify subject info
                # nwb.subject.genotype = "NDNF-flp +/- and Pde1c +/-"
                # nwb.subject.set_modified()


                # nwb.subject.genotype = "NDNF-flp-/- and Pde1c -/-"
                
                io.write(nwb)
            print(f"Successfully processed {csv_file} into {nwb_file}")
            
        except Exception as e:
            print(f"Error processing {csv_file}: {str(e)}")
    else:
        print(f"Warning: No matching NWB file found for {csv_file}")

print("Processing complete!")

Processing column: animal
Error processing 63383_2024-07-10_15-37-51_A_phy_k_manual.csv: column 'matlab_animal' already exists in Units 'units'
Processing column: animal
Error processing 63383_2024-07-13_14-10-05_A_phy_k_manual.csv: column 'matlab_animal' already exists in Units 'units'
Processing column: animal
Error processing 63383_2024-07-13_14-42-37_B_phy_k_manual.csv: column 'matlab_animal' already exists in Units 'units'
Processing column: animal
Error processing 63383_2024-07-13_15-10-36_C_phy_k_manual.csv: column 'matlab_animal' already exists in Units 'units'


C:\Users\sachur\AppData\Roaming\Python\Python311\site-packages\pynwb\ecephys.py:90: UserWarning: ElectricalSeries 'lfp_car': The second dimension of data does not match the length of electrodes. Your data may be transposed.
  warnings.warn("%s '%s': The second dimension of data does not match the length of electrodes. "


Processing column: animal
Error processing 63383_2024-07-15_14-51-19_A_phy_k_manual.csv: column 'matlab_animal' already exists in Units 'units'


C:\Users\sachur\AppData\Roaming\Python\Python311\site-packages\pynwb\ecephys.py:90: UserWarning: ElectricalSeries 'lfp_car': The second dimension of data does not match the length of electrodes. Your data may be transposed.
  warnings.warn("%s '%s': The second dimension of data does not match the length of electrodes. "


Processing column: animal
Error processing 63383_2024-07-15_15-18-02_B_phy_k_manual.csv: column 'matlab_animal' already exists in Units 'units'


...

Processing column: animal
Error processing 66539_2024-12-22_16-57-58_A_phy_k_manual.csv: column 'matlab_animal' already exists in Units 'units'


C:\Users\sachur\AppData\Roaming\Python\Python311\site-packages\pynwb\ecephys.py:90: UserWarning: ElectricalSeries 'lfp_car': The second dimension of data does not match the length of electrodes. Your data may be transposed.
  warnings.warn("%s '%s': The second dimension of data does not match the length of electrodes. "


Processing column: animal
Error processing 66539_2024-12-22_17-23-44_B_phy_k_manual.csv: column 'matlab_animal' already exists in Units 'units'


C:\Users\sachur\AppData\Roaming\Python\Python311\site-packages\pynwb\ecephys.py:90: UserWarning: ElectricalSeries 'lfp_car': The second dimension of data does not match the length of electrodes. Your data may be transposed.
  warnings.warn("%s '%s': The second dimension of data does not match the length of electrodes. "


Processing column: animal
Error processing 66539_2024-12-22_17-49-05_C_phy_k_manual.csv: column 'matlab_animal' already exists in Units 'units'
Processing column: animal
Error processing 66922_2024-09-25_15-39-48_A_phy_k_manual.csv: column 'matlab_animal' already exists in Units 'units'
Processing complete!


In [ ]:

# Create summary DataFrame
summary_df = pd.DataFrame(list(summary_data.values()))
summary_df = summary_df[[
    "Animal ID", "Sex", "Genotype", "Experimenter", "Sessions", "Total Units", "Good Units", 
    "Pyramidal Cells", "Narrow Spike Interneurons", "Wide Spike Interneurons", 
    "Speed Cells", "Place Cells"
]]

# Create files DataFrame with unit tables
files_df = pd.DataFrame(file_data)

# Create all units DataFrame
all_units_df = pd.concat(all_units_list, ignore_index=True)

# Save summary table
summary_output_file = os.path.join(output_path, "animal_summary_table.xlsx")
summary_df.to_excel(summary_output_file, index=False)
print(f"Summary table saved to {summary_output_file}")

# Save files DataFrame as pickle (contains DataFrame objects)
files_output_file = os.path.join(output_path, "files_with_unit_tables.pkl")

# Save all units DataFrame
all_units_output_file = os.path.join(output_path, "all_units_table.pkl")
all_units_df.to_pickle(all_units_output_file)
print(f"All units table saved to {all_units_output_file}")

# Print summaries
print("\nSummary Table:")
print(summary_df)
print("\nFiles DataFrame (without unit_table column):")
print(files_df[['filename', 'animal_id', 'genotype', 'experimenter', 'sex']])
print("\nAll Units DataFrame shape:", all_units_df.shape)
print("First few rows of All Units DataFrame:")
print(all_units_df.head())
print("\nNumber of files processed:", len(files_df))
print("Total number of units:", len(all_units_df))

print("Processing complete!")

Summary table saved to Q:/sachuriga/CR_CA1_paper/tables/animal_summary_table.xlsx


OSError: [Errno 22] Invalid argument

In [6]:
all_units_output_file = os.path.join(output_path, "all_units_table.pkl")
files_df.to_pickle(files_output_file)

In [ ]:
import os
file_path = r'Q:/sachuriga/Temporal/CR_CA1_paper/tables/files_with_unit_tables.pkl'
if os.path.exists(file_path):
    size = os.path.getsize(file_path)
    print(f"File exists. Size: {size} bytes")
else:
    print("File does not exist at the specified path.")

File exists. Size: 1457206786 bytes


In [5]:
import pickle

file_path = r'Q:/sachuriga/Temporal/CR_CA1_paper/tables/files_with_unit_tables.pkl'
try:
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    print("File loaded successfully with pickle.")
except EOFError:
    print("EOFError: The file is empty or corrupted.")
except Exception as e:
    print(f"An error occurred: {e}")

EOFError: The file is empty or corrupted.
